# Init

In [1]:
# To work with files
import os

# Used in funct def stats(match, state, df): 
import numpy as np

# To use DataFrames
import pandas as pd

# To parse time
from datetime import datetime, timedelta 

# Used in funct def comp_angle(row):
import math

# To plot graphs
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib nbagg

# To allow more outputs in Jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Path to left wrist, right wrist Actigraphy and Polysomnography data
path = "C:\\Users\\sigmu\\Desktop\\dataset\\"
path_ACG = path + "Aktigrafie\\"
path_PSG = path + "Polysomnografie\\"
files_ACG = os.listdir(path_ACG)
files_PSG = os.listdir(path_PSG)

# thresholds 10 min & 5 min respectively (divided by 5 ... 5 s epochs)
thr_0 = (10 * 60) / 5
thr = (5 * 60) / 5
thr_3 = (3 * 60) / 5
thr_10 = (10 * 60) / 5

# 5s epochs with 10 minute (first sleep) and 5 min threshold
state_5_10_3 = 'State 5s_10,3'
state_5_10_5 = 'State 5s_10,5'
state_5_10_10 = 'State 5s_10,10'

df_stats_3 = pd.DataFrame(columns=['Participant', 
                                 'TIB', 'SOL', 'TST', 'WASO', 'SWR', 'SE%', 'SFI',
                                 'sensitivity', 'specificity', 'accuracy', 'MCC'],
                        index=[state_5_10_3])

df_stats = pd.DataFrame(columns=['Participant', 
                                 'TIB', 'SOL', 'TST', 'WASO', 'SWR', 'SE%', 'SFI',
                                 'sensitivity', 'specificity', 'accuracy', 'MCC'],
                        index=[state_5_10_5])

df_stats_10 = pd.DataFrame(columns=['Participant', 
                                 'TIB', 'SOL', 'TST', 'WASO', 'SWR', 'SE%', 'SFI',
                                 'sensitivity', 'specificity', 'accuracy', 'MCC'],
                        index=[state_5_10_10])

returns_path_csv = path + "\\Results\\" + "_Stats_" + '.csv'
returns_path_csv_3 = path + "\\Results\\" + "_Stats_3" + '.csv'
returns_path_csv_10 = path + "\\Results\\" + "_Stats_10" + '.csv'
# if file doesn't exist, create new
if ~os.path.exists(returns_path_csv):
    df_stats.to_csv(returns_path_csv)
    df_stats_3.to_csv(returns_path_csv_3)
    df_stats_10.to_csv(returns_path_csv_10)

# Functions

In [4]:
# function to cut start of the DataFrame to match PSG time stamp

def cutStartToMatch(df, df_PSG):
    # Match start
    # If df and PSG day is the same    
    if(df['time stamp'].dt.day[0] == df_PSG['Time [hh:mm:ss]'].dt.day[0]):        
        # If df starts earlier than df_PSG -> cut start of df        
        if(df['time stamp'].dt.hour[0] < df_PSG['Time [hh:mm:ss]'].dt.hour[0]):            
            # Find df index where time matches            
            idx_start = df[(df['time stamp'].dt.hour == df_PSG['Time [hh:mm:ss]'].dt.hour[0]) &
                           (df['time stamp'].dt.minute == df_PSG['Time [hh:mm:ss]'].dt.minute[0]) &
                           (df['time stamp'].dt.second == df_PSG['Time [hh:mm:ss]'].dt.second[0])].index[0]
            # Drop df from 0 to idx_start            
            df.drop(df.index[0:(idx_start)], inplace=True)
        # Else cut start of df_PSG        
        else:
            # Find df_PSG index where time matches            
            idx_start = df_PSG[(df['time stamp'].dt.hour[0] == df_PSG['Time [hh:mm:ss]'].dt.hour) & 
                               (df['time stamp'].dt.minute[0] == df_PSG['Time [hh:mm:ss]'].dt.minute)].index[0]
            # Drop df_PSG from 0 to idx_start            
            df_PSG.drop(df_PSG.index[0:(idx_start)], inplace=True) 
    # Else if df starts day earlier (happens in #45) -> cut start of df    
    elif(df['time stamp'].dt.day[0] < df_PSG['Time [hh:mm:ss]'].dt.day[0]):
        # Find df index where time matches            
        idx_start = df[(df['time stamp'].dt.hour == df_PSG['Time [hh:mm:ss]'].dt.hour[0]) &
                       (df['time stamp'].dt.minute == df_PSG['Time [hh:mm:ss]'].dt.minute[0]) &
                       (df['time stamp'].dt.second == df_PSG['Time [hh:mm:ss]'].dt.second[0])].index[0]
        # Drop df from 0 to idx_start
        df.drop(df.index[0:(idx_start)], inplace=True)
    # Else - not expected (no ACG b4 PSG in current dataset)    
    else:
        print("Something's wrong in cutStartToMatch.")
 
    return

def cutEndToMatch(df, df_PSG): 
    df.reset_index(inplace=True, drop=True)
    df_PSG.reset_index(inplace=True, drop=True)
    df_PSG_len = len(df_PSG.index)-1
    df_len = len(df.index)-1
    
    # Match end
    # If df ends earlier than df_PSG -> cut end of df_PSG
    if(df['time stamp'].dt.hour[df_len] < df_PSG['Time [hh:mm:ss]'].dt.hour[df_PSG_len]):
        idx_end = df_PSG[(df['time stamp'].dt.hour[df_len] == df_PSG['Time [hh:mm:ss]'].dt.hour) &
                         (df['time stamp'].dt.minute[df_len]+1 == df_PSG['Time [hh:mm:ss]'].dt.minute)].index[0]  # + 1 min  
        # Drop df_PSG from df end to end
        df_PSG.drop(df_PSG.index[(idx_end):(len(df_PSG.index))], inplace=True)
    # Else cut end of df
    else:
        idx_end = df[(df['time stamp'].dt.hour == df_PSG['Time [hh:mm:ss]'].dt.hour[df_PSG_len]) &
                     (df['time stamp'].dt.minute == df_PSG['Time [hh:mm:ss]'].dt.minute[df_PSG_len]) &
                     (df['time stamp'].dt.second == df_PSG['Time [hh:mm:ss]'].dt.second[df_PSG_len])].index[0]
        # Drop df from df_PSG end to end
        df.drop(df.index[(idx_end):(len(df.index))], inplace=True)
        
    return
        
# ----------------------------------------------------------------------------------------------------------- 

def comp_angle(row):
    return math.degrees(math.atan(row['z axis [g]'] / (row['x axis [g]']**2 + row['y axis [g]']**2)))

# -----------------------------------------------------------------------------------------------------------   

def inactiv(first_threshold, threshold, column_name, dataframe):
    SleepOnset = 0
    SleepEnd = 0
    SleepFragmentation = 0
    counter = 0
    non_wear_counter = 0
    first_sleep = False
    woke_up = False
    result = []
    for index, value in df['abs angle change'].items():
        counter += 1

        # Non-wear possible solution: if angle change is 0 for at least an hour
        if (value == 0):
            non_wear_counter += 1
            if(non_wear_counter == 720):
                df[state_5_10_5] = "N"
                break
        else:
            non_wear_counter = 0
                
        # Angle change > 5 -> woke up
        if (value > 5):
            counter = 0
            #df.loc[index, clmn_name] = "W"
            # after SO if woken up, add to Sleep Fragmentation
            if(first_sleep):
                if(woke_up == False):
                    SleepFragmentation += 1
                    woke_up = True              
        # After first sleep - shorter threshold    
        elif (counter > threshold) & first_sleep:
            dataframe.loc[index, column_name] = "S"
            # Sleep End (end of sleep period)
            SleepEnd = index
            woke_up = False
        # First sleep - longer threshold 
        elif (counter > first_threshold):
            dataframe.loc[index, column_name] = "S"
            first_sleep = True
            # Sleep Onset (start of sleep period)
            SleepOnset = index   
            
    result.append(SleepOnset)
    result.append(SleepEnd)
    result.append(SleepFragmentation)
    return result
 
# -----------------------------------------------------------------------------------------------------------
    
def stats(match, state, df, df_stats):
    
    # If non-wear
    if(df[state][0] == 'N'):
        print("Accelerometr is not being used.")
        return
    
    # Sleep Onset and last Sleep from inactiv func (as timestamp -> str)
    SOnset = str(latency[0].time())    
    SEnd = str(latency[1].time())
    last = df[state_5_10_5][len(df)-1]
    SEnd_greater = df.index[len(df)-1] < latency[1]
    # If last item in df is sleep and it is less than SEnd (SEnd was created with 5s epochs - could be after last item)
    if ((last == 'S') & SEnd_greater):
        SEnd = str(df.index[len(df)-1].time())
        
    # Sleep Onset Latency: SOnset - start of recording
    SOL = latency[0] - df_PSG['Time [hh:mm:ss]'][0]
    SOL = round(SOL.seconds / 60, 2)
    SOL
    df_stats['SOL'][state] = SOL        
    
    # TIB is presumed to be whole recording after cutStartToMatch, cutEndToMatch
    diff = (df_PSG['Time [hh:mm:ss]'][len(df_PSG)-1] - df_PSG['Time [hh:mm:ss]'][0])
    TIB = diff.seconds / 60
    df_stats['TIB'][state] = TIB
    
    # TST is the duration in minutes of all sleep epochs (when activity count is zero) between Sleep Onset (SO) time and 
    ##sleep end
    TST = (len(df[df[state]=='S'].index.value_counts()) * 30 ) / 60
    df_stats['TST'][state] = TST

    # WASO is the duration in minutes calculated by summing of all wake epochs between SO time and sleep end
    WASO = (len(df[df[state]=='W'].index.value_counts()) * 30 ) / 60 - SOL
    df_stats['WASO'][state] = WASO

    # Sleep Wake Ratio ... the bigger the better sleeper 
    SWR = round(TST / WASO, 4)
    df_stats['SWR'][state] = SWR    
    
    # Sleep Eficiency
    SE = round((TST / TIB)*100, 4)
    df_stats['SE%'][state] = SE
    
    # Sleep Fragmentation is number of intervals scored as "awake" (after sleep onset) relative to the total sleep time in hours
    SFI = round(latency[2] / (TST/60), 2)
    df_stats['SFI'][state] = SFI
    
    #sensitivity (actigraphy = sleep when PSG = sleep), 
    #specificity (actigraphy = wake when PSG = wake), 
    #and accuracy (total proportion correct)
    # Put all states where: into an array
    TP = np.where((df[state] == 'S') & (df['State PSG'] == 'S'))
    FP = np.where((df[state] == 'S') & (df['State PSG'] == 'W'))
    TN = np.where((df[state] == 'W') & (df['State PSG'] == 'W'))
    FN = np.where((df[state] == 'W') & (df['State PSG'] == 'S'))

    # Return array size
    TP_cnt = np.size(TP)
    FP_cnt = np.size(FP)
    TN_cnt = np.size(TN)
    FN_cnt = np.size(FN)

    # Calculate sensitivity, specificity and accuracy
    sens = round((TP_cnt / (TP_cnt + FN_cnt) )*100, 4)
    spec = round((TN_cnt / (TN_cnt + FP_cnt) )*100, 4)
    acc = round(((TP_cnt + TN_cnt) / (TP_cnt + TN_cnt + FP_cnt + FN_cnt) )*100, 4)
    
    # Matthews correlation coefficient: range -1 to 1, perfect is 1, always missclassifies when -1 and 0 is a coin flip
    MCC = round((TP_cnt*TN_cnt-FP_cnt*FN_cnt)/math.sqrt((TP_cnt+FP_cnt)*(TP_cnt+FN_cnt)*(TN_cnt+FP_cnt)*(TN_cnt+FN_cnt)), 2)

    # Add to df
    df_stats['Participant'][state] = participant+'_'+arm
    df_stats['sensitivity'][state] = sens
    df_stats['specificity'][state] = spec    
    df_stats['accuracy'][state] = acc
    df_stats['MCC'][state] = MCC

    return

# -----------------------------------------------------------------------------------------------------------

def datasetStats():
    return
#    acc, sens, spec
#    bias (mean ± standard deviation between patients) in estimated sleep duration

# Iterate through files

In [5]:
%%time

first_File = True

# go through PSG files: eg. mecsleep01_psg.txt (take 1 file and continue)
for file_name_PSG in files_PSG:
    # split on _, take only mecsleep01 and put to uppercase
    participant = file_name_PSG.split("_")[0].upper() 
    # go through ACG files: take left and right arm rec (2 files for 1 PSG file)
    for file_name_ACG in files_ACG: 
        arm = file_name_ACG.split("_")[1]
        if(file_name_ACG.startswith(participant+"_"+arm)):
            print(file_name_ACG)
            df = pd.read_csv(path_ACG+file_name_ACG, names=['time stamp', 'x axis [g]', 'y axis [g]', 'z axis [g]', 
                                                            'light level [lux]', 'button [1/0]','temperature [°C]'],
                             skiprows=100,
                             # might be slightly faster:
                            infer_datetime_format=True, memory_map=True)
            df['time stamp'] = pd.to_datetime(df['time stamp'], format='%Y-%m-%d %H:%M:%S:%f')

            # 1st PSG read -> get recording date
            df_PSG = pd.read_csv(path_PSG + file_name_PSG,
                                infer_datetime_format=True, memory_map=True)
            PSGdate = df_PSG["RemLogic Event Export"][2].split("\t")[1]
            # 2nd PSG read -> parse to datetime
            df_PSG = pd.read_csv(path_PSG + file_name_PSG, sep='\t', skiprows=17,
                                infer_datetime_format=True, memory_map=True)
            df_PSG['Time [hh:mm:ss]'] = PSGdate + " " + df_PSG['Time [hh:mm:ss]']
            df_PSG['Time [hh:mm:ss]'] = pd.to_datetime(df_PSG['Time [hh:mm:ss]'], format='%d/%m/%Y %H:%M:%S')

            # drop not used columns from ACG
            df.drop(columns=['light level [lux]', 'button [1/0]', 'temperature [°C]'], inplace=True, axis=1)

            # drop not used columns from PSG
            # Commented because 'Position' column missing in 34               
            #df_PSG.drop(columns=['Position', 'Event', 'Duration[s]'], inplace=True, axis=1) 

            #-----------------------------------------------------------------------------------------------

            # Drop values so that the ACG and PSG recording is starting and ending at the same time
            cutStartToMatch(df, df_PSG)
            cutEndToMatch(df, df_PSG)

            #-----------------------------------------------------------------------------------------------

            # Set origin from PSG to start resampling
            orig = (df_PSG['Time [hh:mm:ss]'].dt.hour[0]*3600 + 
                    (df_PSG['Time [hh:mm:ss]'].dt.minute[0]*60) +
                    (df_PSG['Time [hh:mm:ss]'].dt.second[0]))
            orig  = pd.Timestamp(orig, unit='s')

            # Resample by 5 second epoch and compute median of x,y,z
            df = df.resample('5S', on='time stamp', kind='timestamp', origin=orig).median().round(decimals=4)

            #-----------------------------------------------------------------------------------------------  

            # Apply func comp_angle
            df['angle'] = df.apply(comp_angle, axis=1).round(decimals=4)

            # Average the angle per 5 sec epoch
            #df['angle'] = df.resample('5S').mean().round(decimals=4)

            # Return absolute difference in angle per column
            df['abs angle change'] = df['angle'].diff().round(decimals=4).abs()
            
# comp-------------------------
            df_backup = df
            
            # New column with all "W" values
            df[state_5_10_5] = "W"

            #-----------------------------------------------------------------------------------------------    

            # Decide inactivity based on the angle            
            # returns list of timestamps for SE, SO
            latency = inactiv(thr_0, thr, state_5_10_5, df)

            #-----------------------------------------------------------------------------------------------

            # Resample ACG to have same number of columns as PSG (30s epochs)
            df = df.resample('30S', origin=orig).interpolate()

            # If PSG has extra value at the end
            if(len(df_PSG) > len(df)):
                df_PSG.drop(df_PSG.index[len(df_PSG)-1], inplace=True)

            # Overwrite State PSG to bi-state 
            df['State PSG'] = np.where((df_PSG['Sleep Stage'] == 'N1')|(df_PSG['Sleep Stage'] == 'N2')|
                                               (df_PSG['Sleep Stage'] == 'N3')|(df_PSG['Sleep Stage'] == 'R'), 'S', 'W')

            # Create boolean column and compare
            match = 'match_' + state_5_10_5
            df[match] = np.where(df[state_5_10_5] == df['State PSG'], True, False)

            #-----------------------------------------------------------------------------------------------

            # Participant according to file name
            df_stats['Participant'][0] = participant+'_'+arm

            # Compute statistics
            match = 'match_' + state_5_10_5
            stats(match, state_5_10_5, df, df_stats)

            #-----------------------------------------------------------------------------------------------

            # Return to files
            df.to_csv(path + "\\Results\\" + "df__" + file_name_ACG)
            if(first_File):
                df_stats.to_csv(returns_path_csv)
#                first_File = False
            else:                
                df_stats.to_csv(returns_path_csv, mode='a', header=False)
                
#comp -----------------------------------------
            df = df_backup
    
            # New column with all "W" values
            df[state_5_10_3] = "W"

            #-----------------------------------------------------------------------------------------------    

            # Decide inactivity based on the angle            
            # returns list of timestamps for SE, SO
            latency = inactiv(thr_0, thr_3, state_5_10_3, df)

            #-----------------------------------------------------------------------------------------------

            # Resample ACG to have same number of columns as PSG (30s epochs)
            df = df.resample('30S', origin=orig).interpolate()

            # If PSG has extra value at the end
            if(len(df_PSG) > len(df)):
                df_PSG.drop(df_PSG.index[len(df_PSG)-1], inplace=True)

            # Overwrite State PSG to bi-state 
            df['State PSG'] = np.where((df_PSG['Sleep Stage'] == 'N1')|(df_PSG['Sleep Stage'] == 'N2')|
                                               (df_PSG['Sleep Stage'] == 'N3')|(df_PSG['Sleep Stage'] == 'R'), 'S', 'W')

            # Create boolean column and compare
            match = 'match_' + state_5_10_3
            df[match] = np.where(df[state_5_10_3] == df['State PSG'], True, False)

            #-----------------------------------------------------------------------------------------------

            # Participant according to file name
            df_stats_3['Participant'][0] = participant+'_'+arm

            # Compute statistics
            match = 'match_' + state_5_10_3
            stats(match, state_5_10_3, df, df_stats_3)

            #-----------------------------------------------------------------------------------------------

            # Return to files
            df.to_csv(path + "\\Results\\" + "df_3_" + file_name_ACG)
            if(first_File):
                df_stats_3.to_csv(returns_path_csv_3)
#                first_File = False
            else:                
                df_stats_3.to_csv(returns_path_csv_3, mode='a', header=False)
                
# comp ---------------------------------------------

            df = df_backup
    
            # New column with all "W" values
            df[state_5_10_10] = "W"

            #-----------------------------------------------------------------------------------------------    

            # Decide inactivity based on the angle            
            # returns list of timestamps for SE, SO
            latency = inactiv(thr_0, thr_10, state_5_10_10, df)

            #-----------------------------------------------------------------------------------------------

            # Resample ACG to have same number of columns as PSG (30s epochs)
            df = df.resample('30S', origin=orig).interpolate()

            # If PSG has extra value at the end
            if(len(df_PSG) > len(df)):
                df_PSG.drop(df_PSG.index[len(df_PSG)-1], inplace=True)

            # Overwrite State PSG to bi-state 
            df['State PSG'] = np.where((df_PSG['Sleep Stage'] == 'N1')|(df_PSG['Sleep Stage'] == 'N2')|
                                               (df_PSG['Sleep Stage'] == 'N3')|(df_PSG['Sleep Stage'] == 'R'), 'S', 'W')

            # Create boolean column and compare
            match = 'match_' + state_5_10_10
            df[match] = np.where(df[state_5_10_10] == df['State PSG'], True, False)

            #-----------------------------------------------------------------------------------------------

            # Participant according to file name
            df_stats_10['Participant'][0] = participant+'_'+arm

            # Compute statistics
            match = 'match_' + state_5_10_10
            stats(match, state_5_10_10, df, df_stats_10)

            #-----------------------------------------------------------------------------------------------

            # Return to files
            df.to_csv(path + "\\Results\\" + "df_10_" + file_name_ACG)
            if(first_File):
                df_stats_10.to_csv(returns_path_csv_10)
                first_File = False
            else:                
                df_stats_10.to_csv(returns_path_csv_10, mode='a', header=False)

            #print("df", df_stats)
            #df_all_stats.append(df_stats)
            #print("df", df_all_stats)

            # Delete existing dataframes?
            #del df
            #del df_PSG
            
# name=['Participant', 
#         'TP_cnt', 'TN_cnt', 'FP_cnt', 'FN_cnt',
#         'TST', 'WASO', 'SWR', 
#         'specificity', 'sensitivity', 'accuracy']            
# df = pd.read_csv(returns_path_csv, names=[
#                  'time stamp', 'x axis [g]', 'y axis [g]', 'z axis [g]', 'light level [lux]', 'button [1/0]','temperature [°C]'], 
#                  skiprows=100)            

MECSLEEP01_left_wrist_012870_2013-06-12_11-40-37.csv
MECSLEEP01_right_wrist_012855_2013-06-11_12-08-25.csv
MECSLEEP02_left_wrist_012859_2013-06-12_12-05-48.csv
MECSLEEP02_right_wrist_012869_2013-06-12_12-02-20.csv
MECSLEEP10_left_wrist_012859_2013-11-01_14-03-32.csv
MECSLEEP14_left_wrist_012870_2013-12-09_10-53-34.csv
MECSLEEP17_left_wrist_012854_2013-12-09_11-37-24.csv
MECSLEEP17_right_wrist_012932_2013-12-09_11-30-44.csv
MECSLEEP21_left_wrist_012854_2014-01-23_14-13-59.csv
MECSLEEP21_right_wrist_012932_2014-01-23_14-07-25.csv
MECSLEEP23_left_wrist_016283_2014-02-03_13-24-04.csv
MECSLEEP23_right_wrist_014883_2014-02-03_13-19-40.csv
MECSLEEP27_left_wrist_012867_2014-02-06_12-49-07.csv
Accelerometr is not being used.
MECSLEEP27_right_wrist_012851_2014-02-06_12-42-54.csv
MECSLEEP28_left_wrist_012856_2014-02-13_11-13-26.csv
MECSLEEP28_right_wrist_012854_2014-02-13_11-10-56.csv
MECSLEEP29_left_wrist_014883_2014-02-13_11-21-35.csv
MECSLEEP29_right_wrist_012865_2014-02-13_11-19-27.csv
MECSLE

In [6]:
stats_3 = pd.read_csv(returns_path_csv_3)
stats = pd.read_csv(returns_path_csv)
stats_10 = pd.read_csv(returns_path_csv_10)
stats_3
stats
stats_10

,Unnamed: 0,Participant,TIB,SOL,TST,WASO,SWR,SE%,SFI,sensitivity,specificity,accuracy,MCC
0,"State 5s_10,3",MECSLEEP01_left,606.5,80.25,442.5,84.25,5.2522,72.9596,3.12,83.8074,60.3333,78.0066,0.43
1,"State 5s_10,3",MECSLEEP01_right,606.5,85.25,434.0,87.75,4.9459,71.5581,3.46,82.2757,61.3333,77.1005,0.42
2,"State 5s_10,3",MECSLEEP02_left,467.0,15.58,340.0,111.92,3.0379,72.8051,4.76,86.0570,60.4478,78.7166,0.47
3,"State 5s_10,3",MECSLEEP02_right,467.0,16.33,346.5,104.67,3.3104,74.1970,3.98,88.9055,62.6866,81.3904,0.53
4,"State 5s_10,3",MECSLEEP10_left,526.5,10.00,390.0,127.00,3.0709,74.0741,3.69,81.2500,58.9474,77.2296,0.35
5,"State 5s_10,3",MECSLEEP14_left,459.0,24.08,37.0,398.42,0.0929,8.0610,45.41,6.3636,91.4163,71.0555,-0.03
6,"State 5s_10,3",MECSLEEP17_left,487.0,46.33,316.5,124.67,2.5387,64.9897,4.74,74.0426,58.8889,69.8462,0.31
7,"State 5s_10,3",MECSLEEP17_right,487.0,46.33,307.5,133.67,2.3004,63.1417,4.88,71.9149,60.0000,68.6154,0.30
8,"State 5s_10,3",MECSLEEP21_left,641.5,17.83,338.0,286.17,1.1811,52.6890,4.97,77.4390,73.2484,75.3894,0.51
9,"State 5s_10,3",MECSLEEP21_right,641.5,22.25,352.5,267.25,1.3190,54.9493,5.11,83.0793,74.5223,78.8941,0.58


,Unnamed: 0,Participant,TIB,SOL,TST,WASO,SWR,SE%,SFI,sensitivity,specificity,accuracy,MCC
0,"State 5s_10,5",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"State 5s_10,5",MECSLEEP29_right,766.0,172.17,417.5,176.83,2.3610,54.5039,2.44,58.6861,48.9387,53.2942,0.08
2,"State 5s_10,5",MECSLEEP31_left,595.0,23.25,404.0,168.25,2.4012,67.8992,3.12,70.7292,44.1558,65.5751,0.13
3,"State 5s_10,5",MECSLEEP31_right,595.0,17.75,448.0,129.75,3.4528,75.2941,2.14,77.2917,33.3333,68.7657,0.10
4,"State 5s_10,5",MECSLEEP32_left,520.0,110.42,251.5,158.58,1.5860,48.3654,3.82,60.1896,70.0980,64.0730,0.30
5,"State 5s_10,5",MECSLEEP32_right,520.0,100.08,213.0,207.42,1.0269,40.9615,6.76,44.7077,64.9510,52.6417,0.10
6,"State 5s_10,5",MECSLEEP34_left,520.0,100.08,213.0,207.42,1.0269,40.9615,6.76,44.7077,64.9510,52.6417,0.10
7,"State 5s_10,5",MECSLEEP34_right,520.0,100.08,213.0,207.42,1.0269,40.9615,6.76,44.7077,64.9510,52.6417,0.10
8,"State 5s_10,5",MECSLEEP35_left,485.0,31.67,136.5,317.33,0.4302,28.1443,7.91,39.8585,80.9872,63.0278,0.23
9,"State 5s_10,5",MECSLEEP35_right,485.0,31.75,140.0,313.75,0.4462,28.8660,6.86,41.9811,81.3528,64.1607,0.26


,Unnamed: 0,Participant,TIB,SOL,TST,WASO,SWR,SE%,SFI,sensitivity,specificity,accuracy,MCC
0,"State 5s_10,10",MECSLEEP01_left,606.5,80.25,304.0,222.75,1.3648,50.1237,3.55,59.5186,78.6667,64.2504,0.33
1,"State 5s_10,10",MECSLEEP01_right,606.5,85.25,283.0,238.75,1.1853,46.6612,3.82,56.2363,82.6667,62.7677,0.34
2,"State 5s_10,10",MECSLEEP02_left,467.0,15.58,210.5,241.42,0.8719,45.0749,3.71,57.4213,85.8209,65.5615,0.39
3,"State 5s_10,10",MECSLEEP02_right,467.0,16.33,236.0,215.17,1.0968,50.5353,3.05,65.5172,86.9403,71.6578,0.47
4,"State 5s_10,10",MECSLEEP10_left,526.5,10.00,272.5,244.50,1.1145,51.7569,2.86,58.4491,78.9474,62.1442,0.29
5,"State 5s_10,10",MECSLEEP14_left,459.0,24.08,0.5,434.92,0.0011,0.1089,240.00,0.0000,99.8569,75.9521,-0.02
6,"State 5s_10,10",MECSLEEP17_left,487.0,46.33,219.5,221.67,0.9902,45.0719,2.19,56.8794,85.9259,64.9231,0.38
7,"State 5s_10,10",MECSLEEP17_right,487.0,46.33,202.0,239.17,0.8446,41.4784,3.56,50.7801,82.9630,59.6923,0.31
8,"State 5s_10,10",MECSLEEP21_left,641.5,17.83,249.5,374.67,0.6659,38.8932,2.40,62.6524,85.9873,74.0654,0.50
9,"State 5s_10,10",MECSLEEP21_right,641.5,22.25,270.0,349.75,0.7720,42.0889,1.78,73.1707,90.4459,81.6199,0.64


In [7]:
stats_3.describe()
stats.describe()
stats_10.describe()

,TIB,SOL,TST,WASO,SWR,SE%,SFI,sensitivity,specificity,accuracy,MCC
count,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,572.935185,40.965741,345.833333,186.636111,2.676700,60.691804,5.488519,72.059583,70.095554,72.869472,0.369630
std,98.358942,43.198560,124.433926,116.061301,1.636230,20.746974,5.996000,20.310516,18.188767,14.841091,0.215459
min,418.500000,10.000000,37.000000,57.080000,0.092900,8.061000,0.000000,6.363600,29.870100,22.719300,-0.030000
25%,497.500000,15.767500,315.750000,112.170000,1.401475,54.288700,3.495000,67.488525,56.880925,68.250075,0.205000
50%,561.000000,21.835000,369.000000,148.790000,2.731500,68.852700,4.315000,77.495000,73.095850,75.439350,0.400000
75%,634.875000,44.747500,441.750000,229.375000,3.579950,74.035600,5.460000,84.311850,86.487900,82.033200,0.517500
max,799.500000,172.170000,516.000000,565.750000,7.848600,83.895100,45.410000,93.888900,97.804600,94.397800,0.790000


,TIB,SOL,TST,WASO,SWR,SE%,SFI,sensitivity,specificity,accuracy,MCC
count,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000
mean,551.486486,34.006757,327.635135,190.344595,1.985124,59.251322,4.456216,67.057278,79.847822,70.044538,0.350000
std,79.617444,35.757407,85.853610,63.264521,0.941224,13.309671,2.011219,13.895198,16.781196,10.940028,0.183576
min,418.500000,10.000000,136.500000,86.080000,0.430200,28.144300,2.140000,38.955800,33.333300,49.625700,0.080000
25%,497.500000,16.580000,304.000000,136.420000,1.492600,52.330000,3.070000,58.686100,64.951000,64.073000,0.230000
50%,534.000000,19.750000,342.500000,176.920000,1.916700,62.349400,3.800000,70.729200,86.187800,70.436800,0.340000
75%,595.000000,31.670000,391.500000,221.670000,2.529000,68.324600,5.190000,77.291700,91.631800,79.686100,0.470000
max,766.000000,172.170000,448.000000,317.330000,4.867600,78.464400,11.760000,87.777800,97.419400,89.375000,0.760000


,TIB,SOL,TST,WASO,SWR,SE%,SFI,sensitivity,specificity,accuracy,MCC
count,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,572.935185,40.965741,225.027778,307.441667,0.855683,39.366428,8.092037,48.684835,86.451737,60.815389,0.294259
std,98.358942,43.198560,93.295353,97.349014,0.488215,15.839202,32.192580,16.608150,13.390687,13.728163,0.175301
min,418.500000,10.000000,0.500000,155.080000,0.001100,0.108900,0.000000,0.000000,37.229400,22.719300,-0.020000
25%,497.500000,15.767500,193.000000,229.247500,0.576675,35.476575,2.845000,43.225625,82.740775,53.363625,0.175000
50%,561.000000,21.835000,241.750000,299.540000,0.829250,41.572550,3.405000,52.056700,90.803050,63.923100,0.300000
75%,634.875000,44.747500,293.875000,366.250000,1.199400,50.473625,5.102500,59.762050,94.973775,70.461075,0.412500
max,799.500000,172.170000,375.500000,602.250000,2.256900,65.543100,240.000000,73.170700,100.000000,92.156900,0.660000


# Continue from file #__